1. Stuff Documents 체인을 사용하여 완전한 RAG 파이프라인을 구현하세요.
2. 체인을 수동으로 구현해야 합니다.
3. 체인에 ConversationBufferMemory를 부여합니다.
4. 이 문서를 사용하여 RAG를 수행하세요: https://gist.github.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223
5. 체인에 다음 질문을 합니다:
    Q1. Aaronson 은 유죄인가요?
    Q2. 그가 테이블에 어떤 메시지를 썼나요?
    Q3. Julia 는 누구인가요?

문서 로드 > 분할 > 임베이딩 > 

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory


In [2]:
llm = ChatOpenAI(
    temperature=0.1,
)

In [3]:
cashe_dir = LocalFileStore(".cache/")

In [4]:
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 600,
    chunk_overlap = 100,
)

In [5]:
loader = UnstructuredFileLoader("document.txt")

In [6]:
docs = loader.load_and_split(text_splitter=splitter)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


In [8]:
embeddings = OpenAIEmbeddings()

cashed_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cashe_dir)


In [13]:
vectorstore = FAISS.from_documents(docs, embeddings)

In [14]:
retriver = vectorstore.as_retriever()


In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
        ("human", "{question}"),
       
    ]
)

In [16]:
memory = ConversationBufferMemory(
    return_messages=True       # 이전 대화 히스토리를 반환하도록 설정
)

In [17]:
chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
)


In [18]:
def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [19]:
invoke_chain("Aaronson 은 유죄인가요?")

content='제가 알기로는 Aaronson이 유죄인지 아닌지에 대한 정보는 제가 가지고 있지 않습니다.'


In [20]:
invoke_chain("그가 테이블에 어떤 메시지를 썼나요?")

content='그가 쓴 메시지는 다음과 같습니다:\nFREEDOM IS SLAVERY\nTWO AND TWO MAKE FIVE\nGOD IS POWER'


In [21]:
invoke_chain("Julia 는 누구인가요?")

content='Julia는 Winston과 사랑을 나누었던 여성입니다.'


In [22]:
print(memory)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Aaronson 은 유죄인가요?'), AIMessage(content='제가 알기로는 Aaronson이 유죄인지 아닌지에 대한 정보는 제가 가지고 있지 않습니다.'), HumanMessage(content='그가 테이블에 어떤 메시지를 썼나요?'), AIMessage(content='그가 쓴 메시지는 다음과 같습니다:\nFREEDOM IS SLAVERY\nTWO AND TWO MAKE FIVE\nGOD IS POWER'), HumanMessage(content='Julia 는 누구인가요?'), AIMessage(content='Julia는 Winston과 사랑을 나누었던 여성입니다.')]) return_messages=True
